In [4]:
import os
os.chdir("/Users/pratham/Desktop/ML:DL Practice")
os.getcwd()

'/Users/pratham/Desktop/ML:DL Practice'

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from os.path import isfile, isdir
% matplotlib inline

In [57]:
train = pd.read_csv("Digit Recognizer/train.csv")
test = pd.read_csv("Digit Recognizer/test.csv")

In [58]:
df = train.copy()
#df = df.drop('label',axis = 1)
#labels = train['label']

In [59]:
df.head(n=6)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
np.shape(df)

(42000, 785)

In [61]:
np.shape(test)

(28000, 784)

In [63]:
# check training data
trainLabelCounts = train['label'].value_counts(sort = False)
trainLabelCounts

0    4132
1    4684
2    4177
3    4351
4    4072
5    3795
6    4137
7    4401
8    4063
9    4188
Name: label, dtype: int64

In [64]:
# cast to numpy array
trainData = train.values[:,1:]
trainLabel = train.values[:,0]
testData = test.values

In [65]:
def preprocessing(data):
    '''
    min-max scaling for every image
    data: numpy array
    output: scaled numpy array
    '''
    minV = 0
    maxV = 255
    data = (data - minV) / (maxV-minV)
    return data
def one_hot_encoding(data, numberOfClass):
    from sklearn import preprocessing
    lb = preprocessing.LabelBinarizer()
    lb.fit(range(numberOfClass))
    return lb.transform(data)

In [66]:
processedTrainData = preprocessing(trainData)
processedTestData = preprocessing(testData)
one_hot_trainLabel = one_hot_encoding(trainLabel, 10)

# Building Neural Network with Tensorflow

In [87]:
def getInputTensor(features, numberOfClass):
    '''
    Create tf.placeholder for input & label
    '''
    inputT = tf.placeholder(dtype = tf.float32, shape = (None, features), name = 'input')
    labelT = tf.placeholder(dtype = tf.float32, shape = (None, numberOfClass), name = 'label')
    keep_prob = tf.placeholder(dtype = tf.float32)
    
    return inputT, labelT, keep_prob

In [88]:
def hiddenLayer(inputT, numberOfNodes):
    '''
    Create hidden layer
    '''
    inputSize = inputT.get_shape().as_list()[1]
    # create weights & biases
    weights = tf.Variable(tf.truncated_normal((inputSize, numberOfNodes)), dtype = tf.float32)
    biases = tf.zeros((numberOfNodes), dtype = tf.float32)
    # output of hidden nodes
    hiddenNodes = tf.add(tf.matmul(inputT, weights), biases)
    hiddenOutput = tf.nn.sigmoid(hiddenNodes)
    
    return hiddenOutput

In [89]:
def outputLayer(hiddenOutput, numberOfClass):
    '''
    Create output layer
    hiddenOutput: output from hidden layer
    numOfClass: number of classes (0~9)
    '''
    inputSize = hiddenOutput.get_shape().as_list()[1]
    # create weights & biases
    weights = tf.Variable(tf.truncated_normal((inputSize, numberOfClass)), dtype = tf.float32)
    biases = tf.zeros((numberOfClass), dtype = tf.float32)
    # output
    output = tf.add(tf.matmul(hiddenOutput, weights), biases)
    
    return output

In [90]:
def build_nn(inputT, numberOfNodes, numberOfClass, keep_prob):
    '''
    build fully connected neural network
    '''
    # fully_connected layers
    fc1 = hiddenLayer(inputT, numberOfNodes)
    output = outputLayer(fc1, numberOfClass)
    
    return output


# Hyperparameters

In [104]:
numberOfNodes = 256
batchSize = 128
numberOfEpoch = 30
learningRate = 0.005
keep_prob_rate = 1.0

In [105]:
# Build Neural Network graph
numberOfClass = 10
imageSize = (28, 28)
features = np.prod(imageSize)
graph = tf.Graph()
tf.reset_default_graph()
with graph.as_default():
    
    # get inputs
    inputT, labelT, keep_prob = getInputTensor(features, numberOfClass)
    
    # build fully-conneted neural network
    logits = build_nn(inputT, numberOfNodes, numberOfClass, keep_prob)
    
    # softmax with cross entropy
    probability = tf.nn.softmax(logits, name = 'probability')
    
    # Cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labelT))
    
    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate = learningRate).minimize(cost)
    
    # accuracy
    correctPrediction = tf.equal(tf.argmax(probability, 1),tf.argmax(labelT, 1))
    accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))

In [ ]:
##Training the model

In [98]:
from sklearn.model_selection import train_test_split

In [99]:
def printResult(epoch,numberOfEpoch,trainLoss,validationLoss,validationAccuracy):
    print("Epoch: {}/{}".format(epoch+1, numberOfEpoch),
         '\tTraining Loss: {:.3f}'.format(trainLoss),
         '\tValidation Loss: {:.3f}'.format(validationLoss),
         '\tAccuracy: {:.2f}%'.format(validationAccuracy*100))

In [106]:
save_dir = './save'
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(numberOfEpoch):
        # training data & validation data
        train_x, val_x, train_y, val_y = train_test_split(trainData, one_hot_trainLabel,\
                                                      test_size = 0.3)   
        # training loss
        for i in range(0, len(train_x), batchSize):
            trainLoss, _, _ = sess.run([cost, probability, optimizer], feed_dict = {
                inputT: train_x[i: i+batchSize],
                labelT: train_y[i: i+batchSize],
                keep_prob: keep_prob_rate   
            })
            
        # validation loss
        valAcc, valLoss = sess.run([accuracy, cost], feed_dict ={
            inputT: val_x,
            labelT: val_y,
            keep_prob: 1.0
        })
        
        
        # print out
        printResult(epoch, numberOfEpoch, trainLoss, valLoss, valAcc)
    # save
    saver = tf.train.Saver()
    saver.save(sess, save_dir)

('Epoch: 1/30', '\tTraining Loss: 1.656', '\tValidation Loss: 1.621', '\tAccuracy: 69.51%')
('Epoch: 2/30', '\tTraining Loss: 0.637', '\tValidation Loss: 1.033', '\tAccuracy: 77.57%')
('Epoch: 3/30', '\tTraining Loss: 0.877', '\tValidation Loss: 0.901', '\tAccuracy: 79.32%')
('Epoch: 4/30', '\tTraining Loss: 0.782', '\tValidation Loss: 0.708', '\tAccuracy: 82.83%')
('Epoch: 5/30', '\tTraining Loss: 0.678', '\tValidation Loss: 0.637', '\tAccuracy: 83.56%')
('Epoch: 6/30', '\tTraining Loss: 0.679', '\tValidation Loss: 0.557', '\tAccuracy: 84.46%')
('Epoch: 7/30', '\tTraining Loss: 0.468', '\tValidation Loss: 0.512', '\tAccuracy: 85.86%')
('Epoch: 8/30', '\tTraining Loss: 0.396', '\tValidation Loss: 0.491', '\tAccuracy: 86.22%')
('Epoch: 9/30', '\tTraining Loss: 0.565', '\tValidation Loss: 0.448', '\tAccuracy: 86.74%')
('Epoch: 10/30', '\tTraining Loss: 0.637', '\tValidation Loss: 0.459', '\tAccuracy: 86.86%')
('Epoch: 11/30', '\tTraining Loss: 0.594', '\tValidation Loss: 0.424', '\tAccur